# Markdown Meeting Notes, Google Doc (Google Docs API)

This Colab converts the provided markdown meeting notes into a formatted Google Doc.


In [ ]:

!pip -q install google-api-python-client google-auth-httplib2 google-auth-oauthlib markdown-it-py reportlab

In [ ]:

from google.colab import auth
auth.authenticate_user()
print("Authenticated.")

In [ ]:

MARKDOWN_NOTES = r'''# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
'''
print(MARKDOWN_NOTES.splitlines()[0])

In [ ]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict

@dataclass
class Paragraph:
    kind: str  
    text: str
    level: int = 0  
    mentions: Tuple[Tuple[int,int], ...] = ()  

MENTION_RE = re.compile(r'@\w+')

def _mention_spans(s: str) -> Tuple[Tuple[int,int], ...]:
    return tuple((m.start(), m.end()) for m in MENTION_RE.finditer(s))

def parse_markdown(md: str) -> List[Paragraph]:
    """Minimal markdown parser for headings, bullets, checkboxes, hr, and footer."""
    lines = md.replace('\r\n','\n').replace('\r','\n').split('\n')

    paragraphs: List[Paragraph] = []
    in_footer = False
    for raw in lines:
        line = raw.rstrip()

        if not line.strip():
            
            paragraphs.append(Paragraph(kind='text', text=''))
            continue

        if line.strip() == '---':
            paragraphs.append(Paragraph(kind='hr', text=''))
            in_footer = True
            continue

        if line.startswith('# '):
            
            title = line[2:].strip()
            
            main, sep, rest = title.partition(' - ')
            paragraphs.append(Paragraph(kind='h1', text=main.strip()))
            if rest.strip():
                paragraphs.append(Paragraph(kind='text', text=rest.strip()))
            continue

        if line.startswith('## '):
            paragraphs.append(Paragraph(kind='h2', text=line[3:].strip()))
            continue

        if line.startswith('### '):
            paragraphs.append(Paragraph(kind='h3', text=line[4:].strip()))
            continue

        
        m_cb = re.match(r'^(\s*)-\s*\[ \]\s+(.*)$', line)
        if m_cb:
            indent = len(m_cb.group(1))
            level = indent // 2
            txt = m_cb.group(2).strip()
            paragraphs.append(Paragraph(kind='checkbox', text=txt, level=level, mentions=_mention_spans(txt)))
            continue

        
        m_b = re.match(r'^(\s*)([-*])\s+(.*)$', line)
        if m_b:
            indent = len(m_b.group(1))
            level = indent // 2
            txt = m_b.group(3).strip()
            kind = 'footer' if in_footer else 'bullet'
            paragraphs.append(Paragraph(kind=kind, text=txt, level=level, mentions=_mention_spans(txt)))
            continue

        
        kind = 'footer' if in_footer else 'text'
        paragraphs.append(Paragraph(kind=kind, text=line.strip(), mentions=_mention_spans(line.strip())))

    
    return paragraphs

paragraphs = parse_markdown(MARKDOWN_NOTES)
print("Parsed paragraphs:", len(parsed))
print("First 8:", parsed[:8])

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth
from typing import Any

DOCS_SCOPES = ["https://www.googleapis.com/auth/documents"]


def create_formatted_doc(paragraphs, doc_title: str = "Product Team Sync") -> str:
    try:
        doc = docs_service.documents().create(body={"title": doc_title}).execute()
        doc_id = doc["documentId"]

        requests = []
        style_reqs = []

        
        cursor = 1

        def _insert_text(text: str):
            nonlocal cursor
            requests.append({"insertText": {"location": {"index": cursor}, "text": text}})
            start = cursor
            cursor += len(text)
            end = cursor
            return start, end

        def _apply_named_style(start: int, end: int, named_style: str):
            style_reqs.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "paragraphStyle": {"namedStyleType": named_style},
                    "fields": "namedStyleType",
                }
            })

        def _apply_indent(start: int, end: int, level: int):
            indent_pts = 18 * max(0, level)  # 0, 18, 36, ...
            style_reqs.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "paragraphStyle": {
                        "indentStart": {"magnitude": indent_pts, "unit": "PT"},
                        "indentFirstLine": {"magnitude": indent_pts, "unit": "PT"},
                    },
                    "fields": "indentStart,indentFirstLine",
                }
            })

        def _apply_footer_style(start: int, end: int):
          
            style_reqs.append({
                "updateTextStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "textStyle": {
                        "italic": True,
                        "fontSize": {"magnitude": 10, "unit": "PT"},
                        "foregroundColor": {
                            "color": {"rgbColor": {"red": 0.45, "green": 0.45, "blue": 0.45}}
                        },
                    },
                    "fields": "italic,fontSize,foregroundColor",
                }
            })

        def _style_mentions(text_start: int, text: str):
            
            import re
            for m in re.finditer(r"@\w+", text):
                m_start = text_start + m.start()
                m_end = text_start + m.end()
                style_reqs.append({
                    "updateTextStyle": {
                        "range": {"startIndex": m_start, "endIndex": m_end},
                        "textStyle": {
                            "bold": True,
                            "foregroundColor": {
                                "color": {"rgbColor": {"red": 0.12, "green": 0.45, "blue": 0.95}}
                            },
                        },
                        "fields": "bold,foregroundColor",
                    }
                })

        for p in paragraphs:
            kind = getattr(p, "kind", None) if not isinstance(p, dict) else p.get("kind")
            text = getattr(p, "text", "") if not isinstance(p, dict) else p.get("text", "")
            level = getattr(p, "level", 0) if not isinstance(p, dict) else p.get("level", 0)

            
            if kind in ("h1", "h2", "h3"):
                block = f"{text}\n"
                start, end = _insert_text(block)
                
                if kind == "h1":
                    _apply_named_style(start, end, "HEADING_1")
                elif kind == "h2":
                    _apply_named_style(start, end, "HEADING_2")
                else:
                    _apply_named_style(start, end, "HEADING_3")
                continue

            
            if kind == "hr":
                
                rule = "—" * 30
                hr_text = f"\n{rule}\n"
                start, end = _insert_text(hr_text)

                line_start = start + 1                 
                line_end = line_start + len(rule)      

                
                style_reqs.append({
                    "updateParagraphStyle": {
                        "range": {"startIndex": line_start, "endIndex": line_end},
                        "paragraphStyle": {"alignment": "CENTER"},
                        "fields": "alignment",
                    }
                })

                
                style_reqs.append({
                    "updateTextStyle": {
                        "range": {"startIndex": line_start, "endIndex": line_end},
                        "textStyle": {
                            "foregroundColor": {
                                "color": {"rgbColor": {"red": 0.6, "green": 0.6, "blue": 0.6}}
                            }
                        },
                        "fields": "foregroundColor",
                    }
                })
                continue

            if kind == "checkbox":
                
                block = f"{text}\n"
                start, end = _insert_text(block)

                
                requests.append({
                    "createParagraphBullets": {
                        "range": {"startIndex": start, "endIndex": end},
                        "bulletPreset": "BULLET_CHECKBOX",
                    }
                })

                _apply_indent(start, end, level)
                _style_mentions(start, block)
                continue

            
            if kind == "bullet":
                block = f"{text}\n"
                start, end = _insert_text(block)

                requests.append({
                    "createParagraphBullets": {
                        "range": {"startIndex": start, "endIndex": end},
                        "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE",
                    }
                })

                _apply_indent(start, end, level)
                _style_mentions(start, block)
                continue

            
            if kind == "footer":
                block = f"{text}\n"
                start, end = _insert_text(block)
                _apply_footer_style(start, end)
                continue

            
            if kind in ("text", None):
                block = f"{text}\n"
                start, end = _insert_text(block)
                _style_mentions(start, block)
                continue
        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={"requests": requests + style_reqs},
        ).execute()

        return f"https://docs.google.com/document/d/{doc_id}/edit"

    except HttpError as e:
        raise RuntimeError(f"Docs batchUpdate failed: {e}") from e
    except Exception as e:
        raise RuntimeError(f"Unexpected error while creating doc: {e}") from e


In [ ]:
from google.colab import auth
auth.authenticate_user()

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

creds, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/documents"])
docs_service = build("docs", "v1", credentials=creds)


print("docs_service OK:", docs_service is not None)
print("paragraphs defined:", "paragraphs" in globals(), "len=", (len(paragraphs) if "paragraphs" in globals() else "N/A"))

try:
    doc_url = create_formatted_doc(paragraphs, doc_title="Product Team Sync")
    print("Created Google Doc:", doc_url)
except Exception as e:
    print("create_formatted_doc failed with error:")
    raise


In [ ]:
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from datetime import datetime

def generate_submission_pdf(
    out_path: str,
    doc_url: str,
    github_repo_url: str = "",
    colab_notebook_url: str = "",
) -> str:
    c = canvas.Canvas(out_path, pagesize=LETTER)
    width, height = LETTER

    x = 0.8 * inch
    y = height - 0.9 * inch

    c.setFont("Helvetica-Bold", 16)
    c.drawString(x, y, "Submission - Markdown to Google Doc")
    y -= 0.35 * inch

    c.setFont("Helvetica", 10)
    c.drawString(x, y, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    y -= 0.35 * inch

    c.setFont("Helvetica-Bold", 11)
    c.drawString(x, y, "Links")
    y -= 0.2 * inch

    def draw_link(label: str, url: str):
        nonlocal y
        c.setFont("Helvetica", 10)
        c.drawString(x, y, f"{label}:")

        c.setFillColorRGB(0.10, 0.35, 0.85)
        shown = url if url else "(add link)"
        c.drawString(x + 110, y, shown)
        c.setFillColorRGB(0, 0, 0)

        if url:
            
            c.linkURL(url, (x + 110, y - 2, x + 110 + 420, y + 10), relative=0)

        y -= 0.22 * inch

    draw_link("Google Doc", doc_url)
    draw_link("GitHub Repo", github_repo_url)
    draw_link("Colab Notebook", colab_notebook_url)

    y -= 0.25 * inch
    c.setFont("Helvetica-Bold", 11)
    c.drawString(x, y, "Notes")
    y -= 0.18 * inch

    c.setFont("Helvetica", 10)
    notes = [
        "This PDF includes links to the generated Google Doc and the code artifacts.",
        "If a link field is blank, paste the URL and re-run this cell.",
    ]
    for n in notes:
        c.drawString(x, y, "- " + n)
        y -= 0.18 * inch

    c.showPage()
    c.save()
    return out_path


DOC_URL = doc_url
GITHUB_REPO_URL = "https://github.com/dhruvpandoh/md-to-gdoc"     
COLAB_NOTEBOOK_URL = "https://colab.research.google.com/drive/1e3De3X5J_hJaSJ3gQBrYUcjvt5jZslx9?usp=sharing"  

pdf_path = "/content/submission_links.pdf"
generate_submission_pdf(
    pdf_path,
    doc_url=DOC_URL,
    github_repo_url=GITHUB_REPO_URL,
    colab_notebook_url=COLAB_NOTEBOOK_URL,
)

print("Wrote:", pdf_path)
print("Download it from the Colab file browser (left panel) -> submission_links.pdf")
